In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os
%matplotlib inline
os.environ["PATH"] += os.pathsep + '.../Anaconda3/Lib/site-packages/sphinx/templates/graphviz'


df = pd.read_csv('.../Humana-Mays/Features.csv')
test = pd.read_csv('.../Humana-Mays/HoldoutData.csv')

In [ ]:
df.set_index('id')

In [ ]:
test.set_index('id')

In [ ]:
X = df.iloc[:,2:17]
y = df.iloc[:,1]
Z = test.iloc[:,1:16]
seed = 42
val_size = 0.4
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size, random_state = seed)
train_dmatrix = xgb.DMatrix (data = X_train, label = y_train)
val_dmatrix = xgb.DMatrix (data = X_val, label = y_val)
test_dmatrix = xgb.DMatrix (data = Z)


num_round = 100
evallist = [(val_dmatrix, 'eval'), (train_dmatrix, 'train')]
param = {'max_depth': 6,
         'booster': 'gbtree' ,
         'eta': 0.1,
         'objective':'binary:logistic',
         'subsample': 1,
         'colsample_bytree': 0.8}
#num_round, evallist
gaia = xgb.train(param, train_dmatrix, evals=evallist, num_boost_round=num_round, early_stopping_rounds=20)

In [ ]:
another_dmatrix = xgb.DMatrix (data = X)

y_predict = gaia.predict(another_dmatrix)
print(y_predict)
np.savetxt('FeatureRisk.csv', y_predict, delimiter = ',')

In [ ]:
atriskpatients = y_predict > 0.5
print(atriskpatients)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_val, y_pred=atriskpatients))

In [ ]:
xgb.plot_importance(gaia)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(y_true=y_val, y_pred=atriskpatients, classes=[0, 1],
                      title='Confusion matrix, without normalization')

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, atriskpatients)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
y_predict = gaia.predict(test_dmatrix)
np.savetxt('HoldoutPredict.csv', y_predict, delimiter = ',')

In [ ]:
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train, y_train)

print(xg_reg.feature_importances_)
xgb.plot_importance(xg_reg)
plt.pyplot.show()

In [ ]:
preds = xg_reg.predict(data_dmatrix)
print(preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
#predictions = [round(value for value in preds)]
#accuracy = accuracy_score(y_test, preds)
#print('Accuracy: % 2f%%' % (accuracy * 100.0))

In [ ]:
def auc(m, train, test):
      return (metrics.roc_auc_score(y_train, m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test, m.predict_proba(test)[:,1]))

In [ ]:
auc(gaia, X_train, X_val)